In [1]:
import xarray as xr
import numpy as np
import glob
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
from datetime import datetime, timezone
import pandas as pd

In [2]:
C1 = 1.19104*10**(-5)  # in [mW (cm−1)−4 m-2 sr−1]
C2 = 1.43877  # in [K cm]

CHANNEL_NAME = {"channel_1": "VIS 0.6", 
                "channel_2": "VIS 0.8", 
                "channel_3": "NIR 1.6", 
                "channel_4": "IR 3.9", 
                "channel_5": "WV 6.2", 
                "channel_6": "WV 7.3", 
                "channel_7": "IR 8.7", 
                "channel_8": "IR 9.7 - O3", 
                "channel_9": "IR 10.8", 
                "channel_10": "IR 12.0", 
                "channel_11": "IR 13.4 - CO2", }
# in [cm−1]
VC = {'MSG1': {"channel_4": 2567.330, "channel_5": 1598.103, "channel_6": 1362.081, "channel_7": 1149.069, 
                "channel_8": 1034.343, "channel_9": 930.647, "channel_10": 839.660, "channel_11": 752.387
                }, 
      'MSG2': {"channel_4": 2568.832, "channel_5": 1600.548, "channel_6": 1360.330, "channel_7": 1148.620, 
                "channel_8": 1035.289, "channel_9": 931.700, "channel_10": 836.445, "channel_11": 751.792
                }, 
      'MSG3': {"channel_4": 2547.771, "channel_5": 1595.621, "channel_6": 1360.377, "channel_7": 1148.130, 
                "channel_8": 1034.715, "channel_9": 929.842, "channel_10": 838.659, "channel_11": 750.653
                }, 
      'MSG4': {"channel_4": 2555.280, "channel_5": 1596.080, "channel_6": 1361.748, "channel_7": 1147.433, 
                "channel_8": 1034.851, "channel_9": 931.122, "channel_10": 839.113, "channel_11": 748.585
                }, }
# unitless
ALPHA = {'MSG1': {"channel_4": 0.9956, "channel_5": 0.9962, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, 
         'MSG2': {"channel_4": 0.9954, "channel_5": 0.9963, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, 
         'MSG3': {"channel_4": 0.9915, "channel_5": 0.9960, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9982
                   }, 
         'MSG4': {"channel_4": 0.9916, "channel_5": 0.9959, "channel_6": 0.9990, "channel_7": 0.9996, 
                   "channel_8": 0.9998, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, }
# in [K]
BETA = {'MSG1': {"channel_4": 3.410, "channel_5": 2.218, "channel_6": 0.478, "channel_7": 0.179, ''
                  "channel_8": 0.060, "channel_9": 0.625, "channel_10": 0.397, "channel_11": 0.578
                  },
        'MSG2': {"channel_4": 3.438, "channel_5": 2.185, "channel_6": 0.470, "channel_7": 0.179, 
                  "channel_8": 0.056, "channel_9": 0.640, "channel_10": 0.408, "channel_11": 0.561
                  },
        'MSG3': {"channel_4": 2.9002, "channel_5": 2.0337, "channel_6": 0.4340, "channel_7": 0.1714, 
                  "channel_8": 0.0527, "channel_9": 0.6084, "channel_10": 0.3882, "channel_11": 0.5390
                  },
        'MSG4': {"channel_4": 2.9438, "channel_5": 2.0780, "channel_6": 0.4929, "channel_7": 0.1731, 
                  "channel_8": 0.0597, "channel_9": 0.6256, "channel_10": 0.4002, "channel_11": 0.5635
                  }, }

# %%
#############
############# look up tables for calculating reflectances
#############
# constants taken from website: 
# https://eumetsatspace.atlassian.net/wiki/spaces/DSDT/pages/1537277953/MSG15+radiances+conversion+to+BT+and+Reflectances
# and from https://www-cdn.eumetsat.int/files/2020-04/pdf_msg_seviri_rad2refl.pdf

IRRAD = {'MSG1': {"channel_1": 65.2296, "channel_2": 73.0127, "channel_3": 62.3715},
         'MSG2': {"channel_1": 65.2065, "channel_2": 73.1869, "channel_3": 61.9923},
         'MSG3': {"channel_1": 65.5148, "channel_2": 73.1807, "channel_3": 62.0208}, 
         'MSG4': {"channel_1": 65.2656, "channel_2": 73.1692, "channel_3": 61.9416}, }


# %%
class ir_channel:
    """
    class that calls channel specific constants from look up tables above
    """
    def __init__(self, satellite, channel):

        self.name = CHANNEL_NAME[channel]
        self.vc = VC[satellite][channel]  # wavenumber in [cm−1]
        self.alpha = ALPHA[satellite][channel]  # unitless
        self.beta = BETA[satellite][channel]  # in [K]

class vis_nir_channel:
    def __init__(self, satellite, channel):
        
        self.name = CHANNEL_NAME[channel]
        self.irrad = IRRAD[satellite][channel]  # irradiance at 1AU in [mW·m-2·(cm-1)-1]

class MSG_satellite:
    def __init__(self, name):
        self.name =  name

    def _get_channel(self, channel_number):
        # return vis/nir or ir channel depending on channel number
        if channel_number <=3:
            return vis_nir_channel(satellite=self.name, channel=f"channel_{channel_number}")
        else:
            return ir_channel(satellite=self.name, channel=f"channel_{channel_number}")

    def rad_2_tb(self, channel_number, radiances):
        # error handling here:
        # TODO: raise exception when given incorrect channel_number, must be >=4

        # get constants for given channel
        channel_consts = self._get_channel(channel_number)

        # converting radiance to brightness temperature [K] with simplified equation
        numerator = C2 * channel_consts.vc
        fraction = C1 * channel_consts.vc**3 / radiances + 1
        denominator = channel_consts.alpha * (np.log(fraction))
        tb = numerator / denominator - channel_consts.beta / channel_consts.alpha  ## [K]
        return tb
    
    def _d(t):
        # Sun-Earth distance in AU at time t
        return None
    
    def _solar_zenith_angle(t, lon, lat):
        # Solar Zenith Angle in Radians at time t and location x
        return None

    def rad_2_refl(self, channel_number, radiances, t, lon, lat):
        # error handling here:
        # TODO: raise exception when given incorrect channel_number, must be <= 3

        # get constants for given channel
        channel_consts = self._get_channel(channel_number)

        numerator = np.pi * radiances * self._d(t)**2
        denominator = channel_consts.irrad * np.cos(self._solar_zenith_angle(t, lon, lat))

# %%
def radiances_2_brightnesstemp_and_reflectances(radiances, channel_number, satellite_name):
    ## radiances in [mW m−2 sr−1 (cm−1)−1)]
    # TODO: add constraint to channel_number (must be >= 4)

    # access correct satellite 
    satellite = MSG_satellite(satellite_name)
    if channel_number <= 3:
        print("not implemented yet for visible and near-infrared")

    elif channel_number >= 4 and channel_number < 12:
        # get brightness temp fro given channel
        return satellite.rad_2_tb(channel_number, radiances)
    
    else:
        print(f"This channel does not exist for satellite {satellite_name}")
        # TODO: raise exception

In [3]:
nc_file_loc = '/p/scratch/exaww/chatterjee1/msg_warmworld/files/'
log_file = nc_file_loc + "processed_files_log.txt"
nan_crop_file = nc_file_loc + "nan_files_log.txt"

months = {
    4:'04/',
    5:'05/',
    6:'06/',
    7:'07/',
    8:'08/',
    9:'09/',
}

## recap data structure

In [4]:
data = xr.open_dataset(nc_file_loc + '04/' + 'HRSEVIRI_20230429T234509Z_20230429T235740Z_epct_ea1b6dd3_PC.nc')
data

<xarray.Dataset>
Dimensions:     (lat: 665, lon: 958)
Coordinates:
  * lat         (lat) float64 30.02 30.06 30.09 30.13 ... 54.91 54.94 54.98
  * lon         (lon) float64 -3.481 -3.444 -3.406 -3.368 ... 32.41 32.44 32.48
Data variables:
    crs         |S1 ...
    channel_1   (lat, lon) float32 ...
    channel_2   (lat, lon) float32 ...
    channel_3   (lat, lon) float32 ...
    channel_4   (lat, lon) float32 ...
    channel_5   (lat, lon) float32 ...
    channel_6   (lat, lon) float32 ...
    channel_7   (lat, lon) float32 ...
    channel_8   (lat, lon) float32 ...
    channel_9   (lat, lon) float32 ...
    channel_10  (lat, lon) float32 ...
    channel_11  (lat, lon) float32 ...
Attributes: (12/45)
    Conventions:                    CF-1.5
    ch01_cal:                       -1.114656032994e+00 2.185600064695e-02
    ch02_cal:                       -1.465775717050e+00 2.874070033431e-02
    ch03_cal:                       -1.211260244250e+00 2.375020086765e-02
    ch04_cal:                       -1.865920103496e-01 3.658666869601e-03
    ch05_cal:                       -4.242236706827e-01 8.318111189856e-03
    ...                             ...
    EPCT_roi:                       custom
    EPCT_sensing_start:             null
    EPCT_sensing_stop:              null
    EPCT_software_name:             DataTailor
    EPCT_software_version:          3.4.0
    EPCT_stretch_method:            null

In [8]:
data.lon.min(), data.lon.max()

(<xarray.DataArray 'lon' ()> Size: 8B
 array(-3.48121086),
 <xarray.DataArray 'lon' ()> Size: 8B
 array(32.48121086))

In [14]:
data.lat.min(), data.lat.max()

(<xarray.DataArray 'lat' ()>
 array(30.01879699),
 <xarray.DataArray 'lat' ()>
 array(54.98120301))

In [7]:
data.lon

<xarray.DataArray 'lon' (lon: 958)> Size: 8kB
array([-3.481211, -3.443633, -3.406054, ..., 32.406054, 32.443633, 32.481211])
Coordinates:
  * lon      (lon) float64 8kB -3.481 -3.444 -3.406 -3.368 ... 32.41 32.44 32.48
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east

In [10]:
#original trained data
data.lon[252:823].values.min(),data.lon[252:823].values.max()

(5.988517745302714, 27.40814196242171)

In [4]:
# Fixed point
'''
target_lat_juelich, target_lon_juelich = 50.9224, 6.3639
target_lat_lin, target_lon_lin = 52.210, 14.122
target_lat_warsaw, target_lon_warsaw = 52.229, 21.012
target_lat_vienna, target_lon_vienna = 48.2081, 16.3713
target_lat_bourges, target_lon_bourges = 47.0812,2.3980 
target_lat_zaragoza, target_lon_zaragoza = 41.6474, -0.8861
target_lat_sirta, target_lon_sirta = 48.717, 2.208
target_lat_cabauw, target_lon_cabauw = 51.9653, 4.8979
'''
#target_lat_hyytiala, target_lon_hyytiala = 61.8417, 24.2896 # NOTE: not possible as the max lat in the data is 54.98120301
target_lat_nuremberg, target_lon_nuremberg = 49.4543, 11.0746

crop_size = 128
half_crop = crop_size // 2

sample_counter = 0
all_crops_nuremberg = [] #all_crops_sirta = [] #all_crops_juelich, all_crops_lin, all_crops_warsaw, all_crops_vienna, all_crops_bourges, all_crops_zargoza = [],[],[],[],[],[]
all_lats_nuremberg = [] #all_lats_sirta = [] #all_lats_juelich, all_lats_lin, all_lats_warsaw, all_lats_vienna, all_lats_bourges, all_lats_zargoza = [],[],[],[],[],[]
all_lons_nuremberg = [] #all_lons_sirta = [] #all_lons_juelich, all_lons_lin, all_lons_warsaw, all_lons_vienna, all_lons_bourges, all_lons_zargoza = [],[],[],[],[],[]
all_times_nuremberg = [] #all_times_sirta = [] #all_times_juelich, all_times_lin, all_times_warsaw, all_times_vienna, all_times_bourges, all_times_zargoza = [],[],[],[],[],[]

first_write = True

for _, key in enumerate(months.keys()):
    loc = nc_file_loc + months[key]
    nc_filepattern = "HRSEVIRI_2023*_PC.nc"
    nc_files = sorted(glob.glob(loc + nc_filepattern))

    for i, file in tqdm(enumerate(nc_files), desc=f"Processing {months[key]}", total=len(nc_files)):

        # Log file
        with open(log_file, 'a') as log:
            log.write(f"{file}\n")

        data = xr.open_dataset(file)
        satellite_name = data.EPCT_product_name.split('-')[0]
        timestamp = data.EPCT_product_name.split('A-')[1].split('.')[0]

        # Use full lat/lon range (adjust slices if necessary)
        lat_full = data.lat.values
        lon_full = data.lon.values
        radiances = data["channel_9"].values

        bt_data = radiances_2_brightnesstemp_and_reflectances(radiances, 9, satellite_name)

        '''
        ############################################################ JUELICH
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_juelich).argmin()
        lon_idx = np.abs(lon_full - target_lon_juelich).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_juelich.append(crop)
        all_lats_juelich.append(lat_full[start_y:end_y])
        all_lons_juelich.append(lon_full[start_x:end_x])
        all_times_juelich.append(timestamp)

        ################################################################# LIN
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_lin).argmin()
        lon_idx = np.abs(lon_full - target_lon_lin).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_lin.append(crop)
        all_lats_lin.append(lat_full[start_y:end_y])
        all_lons_lin.append(lon_full[start_x:end_x])
        all_times_lin.append(timestamp)

        ################################################################# WARSAW
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_warsaw).argmin()
        lon_idx = np.abs(lon_full - target_lon_warsaw).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_warsaw.append(crop)
        all_lats_warsaw.append(lat_full[start_y:end_y])
        all_lons_warsaw.append(lon_full[start_x:end_x])
        all_times_warsaw.append(timestamp)

        ################################################################# VIENNA
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_vienna).argmin()
        lon_idx = np.abs(lon_full - target_lon_vienna).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_vienna.append(crop)
        all_lats_vienna.append(lat_full[start_y:end_y])
        all_lons_vienna.append(lon_full[start_x:end_x])
        all_times_vienna.append(timestamp)

        ################################################################# BOURGES
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_bourges).argmin()
        lon_idx = np.abs(lon_full - target_lon_bourges).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_bourges.append(crop)
        all_lats_bourges.append(lat_full[start_y:end_y])
        all_lons_bourges.append(lon_full[start_x:end_x])
        all_times_bourges.append(timestamp)

        ################################################################# ZARGOZA
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_zaragoza).argmin()
        lon_idx = np.abs(lon_full - target_lon_zaragoza).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_zargoza.append(crop)
        all_lats_zargoza.append(lat_full[start_y:end_y])
        all_lons_zargoza.append(lon_full[start_x:end_x])
        all_times_zargoza.append(timestamp)

        ############################################################ SIRTA
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_sirta).argmin()
        lon_idx = np.abs(lon_full - target_lon_sirta).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_sirta.append(crop)
        all_lats_sirta.append(lat_full[start_y:end_y])
        all_lons_sirta.append(lon_full[start_x:end_x])
        all_times_sirta.append(timestamp)
        
        
        ############################################################ CABAUW
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_cabauw).argmin()
        lon_idx = np.abs(lon_full - target_lon_cabauw).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_cabauw.append(crop)
        all_lats_cabauw.append(lat_full[start_y:end_y])
        all_lons_cabauw.append(lon_full[start_x:end_x])
        all_times_cabauw.append(timestamp)
        '''
         ############################################################ nuremberg
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_nuremberg).argmin()
        lon_idx = np.abs(lon_full - target_lon_nuremberg).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_nuremberg.append(crop)
        all_lats_nuremberg.append(lat_full[start_y:end_y])
        all_lons_nuremberg.append(lon_full[start_x:end_x])
        all_times_nuremberg.append(timestamp)
        
        '''
        ############################################################ HYYTIALA
        # Find closest pixel index to the target location
        lat_idx = np.abs(lat_full - target_lat_hyytiala).argmin()
        lon_idx = np.abs(lon_full - target_lon_hyytiala).argmin()

        # Compute start and end indices
        start_y = max(0, lat_idx - half_crop)
        end_y = start_y + crop_size
        start_x = max(0, lon_idx - half_crop)
        end_x = start_x + crop_size

        # Check if crop goes out of bounds
        if end_y > bt_data.shape[0] or end_x > bt_data.shape[1]:
            with open(nan_crop_file, 'a') as log:
                log.write(f"Out of bounds for file: {file}\n")
            continue

        # Extract crop
        crop = bt_data[start_y:end_y, start_x:end_x]

        if np.isnan(crop).any():
            with open(nan_crop_file, 'a') as log:
                log.write(f"{file} contains NaN in center crop\n")
            continue

        # Store crop and coordinate slices
        all_crops_hyytiala.append(crop)
        all_lats_hyytiala.append(lat_full[start_y:end_y])
        all_lons_hyytiala.append(lon_full[start_x:end_x])
        all_times_hyytiala.append(timestamp)
        
        '''

        sample_counter += 1





In [20]:
data.EPCT_product_name

'MSG3-SEVI-MSG15-0100-NA-20230930235741.625000000Z-NA.nat'

In [21]:
timestamp

'20230930235741'

In [22]:
data

<xarray.Dataset>
Dimensions:     (lat: 665, lon: 958)
Coordinates:
  * lat         (lat) float64 30.02 30.06 30.09 30.13 ... 54.91 54.94 54.98
  * lon         (lon) float64 -3.481 -3.444 -3.406 -3.368 ... 32.41 32.44 32.48
Data variables:
    crs         |S1 ...
    channel_1   (lat, lon) float32 ...
    channel_2   (lat, lon) float32 ...
    channel_3   (lat, lon) float32 ...
    channel_4   (lat, lon) float32 ...
    channel_5   (lat, lon) float32 ...
    channel_6   (lat, lon) float32 ...
    channel_7   (lat, lon) float32 ...
    channel_8   (lat, lon) float32 ...
    channel_9   (lat, lon) float32 nan 96.78 97.19 97.6 ... 33.22 34.24 33.83
    channel_10  (lat, lon) float32 ...
    channel_11  (lat, lon) float32 ...
Attributes: (12/45)
    Conventions:                    CF-1.5
    ch01_cal:                       -1.114656032994e+00 2.185600064695e-02
    ch02_cal:                       -1.465775717050e+00 2.874070033431e-02
    ch03_cal:                       -1.211260244250e+00 2.375020086765e-02
    ch04_cal:                       -1.865920103496e-01 3.658666869601e-03
    ch05_cal:                       -4.242236706827e-01 8.318111189856e-03
    ...                             ...
    EPCT_roi:                       custom
    EPCT_sensing_start:             null
    EPCT_sensing_stop:              null
    EPCT_software_name:             DataTailor
    EPCT_software_version:          3.4.0
    EPCT_stretch_method:            null

In [5]:
'''
# Convert to arrays
all_crops_np = np.array(all_crops_juelich)
all_lats_np = np.array(all_lats_juelich)
all_lons_np = np.array(all_lons_juelich)
all_times_np = np.array(all_times_juelich)

print('Juelich', all_crops_np.shape)

ds = xr.Dataset(
    {
        "sample_juelich_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_juelichcrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')

######    

all_crops_np = np.array(all_crops_lin)
all_lats_np = np.array(all_lats_lin)
all_lons_np = np.array(all_lons_lin)
all_times_np = np.array(all_times_lin)

print('Lindenberg', all_crops_np.shape)

ds = xr.Dataset(
    {
        "sample_lin_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_lincrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')

########

all_crops_np = np.array(all_crops_warsaw)
all_lats_np = np.array(all_lats_warsaw)
all_lons_np = np.array(all_lons_warsaw)
all_times_np = np.array(all_times_warsaw)

print('Warsaw', all_crops_np.shape)

ds = xr.Dataset(
    {
        "sample_warsaw_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_warsawcrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')

##############
all_crops_np = np.array(all_crops_vienna)
all_lats_np = np.array(all_lats_vienna)
all_lons_np = np.array(all_lons_vienna)
all_times_np = np.array(all_times_vienna)

print('Vienna', all_crops_np.shape)


ds = xr.Dataset(
    {
        "sample_vienna_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_viennacrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')
################


all_crops_np = np.array(all_crops_bourges)
all_lats_np = np.array(all_lats_bourges)
all_lons_np = np.array(all_lons_bourges)
all_times_np = np.array(all_times_bourges)

print('Bourges', all_crops_np.shape)

ds = xr.Dataset(
    {
        "sample_bourges_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_bourgescrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')

################

all_crops_np = np.array(all_crops_zargoza)
all_lats_np = np.array(all_lats_zargoza)
all_lons_np = np.array(all_lons_zargoza)
all_times_np = np.array(all_times_zargoza)

print('Zargoza', all_crops_np.shape)

ds = xr.Dataset(
    {
        "sample_zargoza_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_zargozacrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')

#################
# Convert to arrays
all_crops_np = np.array(all_crops_sirta)
all_lats_np = np.array(all_lats_sirta)
all_lons_np = np.array(all_lons_sirta)
all_times_np = np.array(all_times_sirta)

print('Sirta', all_crops_np.shape)

ds = xr.Dataset(
    {
        "sample_sirta_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_sirtacrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')
    
####################

# Convert to arrays
all_crops_np = np.array(all_crops_cabauw)
all_lats_np = np.array(all_lats_cabauw)
all_lons_np = np.array(all_lons_cabauw)
all_times_np = np.array(all_times_cabauw)

print('Cabauw', all_crops_np.shape)

ds = xr.Dataset(
    {
        "sample_cabauw_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_cabauwcrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')
'''
####################

# Convert to arrays
all_crops_np = np.array(all_crops_nuremberg)
all_lats_np = np.array(all_lats_nuremberg)
all_lons_np = np.array(all_lons_nuremberg)
all_times_np = np.array(all_times_nuremberg)

print('Nuremberg', all_crops_np.shape)

ds = xr.Dataset(
    {
        "sample_nuremberg_data": (["sample", "y", "x"], all_crops_np)
    },
    coords={
        "sample": (["sample"], np.arange(len(all_crops_np))),
        "lat": (["sample", "y"], all_lats_np),
        "lon": (["sample", "x"], all_lons_np),
        "time": (["sample"], all_times_np)
    }
)

output_file = "/p/project/exaww/chatterjee1/dataset/warmworld_datasets/" + "msgobs_108_nurembergcrops.nc"

# Write to file
if first_write:
    ds.to_netcdf(output_file, mode='w')

Nuremberg (18567, 128, 128)


### Checking whether data prepared makes sense

In [18]:
data = xr.open_dataset("/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_warsawcrops.nc")
data.sample_warsaw_data.min(), data.sample_warsaw_data.max()

(<xarray.DataArray 'sample_warsaw_data' ()> Size: 8B
 array(180.00010681),
 <xarray.DataArray 'sample_warsaw_data' ()> Size: 8B
 array(306.00558472))

In [19]:
data = xr.open_dataset("/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_lincrops.nc")
data.sample_lin_data.min(), data.sample_lin_data.max()

(<xarray.DataArray 'sample_lin_data' ()> Size: 8B
 array(179.1197052),
 <xarray.DataArray 'sample_lin_data' ()> Size: 8B
 array(308.07324219))

In [20]:
data = xr.open_dataset("/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_juelichcrops.nc")
data.sample_juelich_data.min(), data.sample_juelich_data.max()

(<xarray.DataArray 'sample_juelich_data' ()> Size: 8B
 array(201.1546936),
 <xarray.DataArray 'sample_juelich_data' ()> Size: 8B
 array(308.64157104))

In [21]:
data = xr.open_dataset("/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_bourgescrops.nc")
data.sample_bourges_data.min(), data.sample_bourges_data.max()

(<xarray.DataArray 'sample_bourges_data' ()> Size: 8B
 array(196.97769165),
 <xarray.DataArray 'sample_bourges_data' ()> Size: 8B
 array(313.86401367))

In [22]:
data = xr.open_dataset("/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_viennacrops.nc")
data.sample_vienna_data.min(), data.sample_vienna_data.max()

(<xarray.DataArray 'sample_vienna_data' ()> Size: 8B
 array(181.69418335),
 <xarray.DataArray 'sample_vienna_data' ()> Size: 8B
 array(309.4329834))

In [23]:
data = xr.open_dataset("/p/project/exaww/chatterjee1/dataset/" + "msgobs_108_zargozacrops.nc")
data.sample_zargoza_data.min(), data.sample_zargoza_data.max()

(<xarray.DataArray 'sample_zargoza_data' ()> Size: 8B
 array(178.21508789),
 <xarray.DataArray 'sample_zargoza_data' ()> Size: 8B
 array(324.54244995))

## automating the above code for channel 9 only

In [10]:
'''
nc_file_loc = '/p/scratch/exaww/chatterjee1/msg_warmworld/files/'
log_file = nc_file_loc + "processed_files_log.txt"
nan_crop_file = nc_file_loc + "nan_files_log.txt"
output_dir = "/p/project/exaww/chatterjee1/dataset/warmworld_datasets/"
os.makedirs(output_dir, exist_ok=True)

# Months
months = {
    4: '04/', 5: '05/', 6: '06/', 7: '07/', 8: '08/', 9: '09/',
}

# Parameters
channel_keys = [f"channel_{i}" for i in range(9, 10)]  # Channels 4-11
crop_size = 128
half = crop_size // 2

# Site locations
sites = {
    #"juelich": (50.9224, 6.3639),
    #"lindenberg": (52.210, 14.122),
    #"warsaw": (52.229, 21.012),
    #"vienna": (48.2081, 16.3713),
    #"bourges": (47.0812, 2.3980),
    #"zaragoza": (41.6474, -0.8861),
    #"sirta": (48.717, 2.208),
    #"cabauw": (51.9653, 4.8979),
    #"nuremberg": (49.4543, 11.0746),
    "aurillac": (44.9332, 2.4483),
    "dresden": (51.0504, 13.7373),
}

# Initialize storage
all_crops = {site: {ch: [] for ch in range(9, 10)} for site in sites}
all_lats = {site: [] for site in sites}
all_lons = {site: [] for site in sites}
all_times = {site: [] for site in sites}

# Utility functions
def get_crop_center_indices_1d(lat_1d, lon_1d, lat_pt, lon_pt):
    iy = np.abs(lat_1d - lat_pt).argmin()
    ix = np.abs(lon_1d - lon_pt).argmin()
    return iy, ix

def radiances_2_brightnesstemp_and_reflectances(radiances, channel_number, satellite_name):
    satellite = MSG_satellite(satellite_name)
    if 4 <= channel_number < 12:
        return satellite.rad_2_tb(channel_number, radiances)
    else:
        raise ValueError(f"Unsupported channel {channel_number} for {satellite_name}")

# Processing loop
for key in months:
    loc = nc_file_loc + months[key]
    nc_filepattern = "HRSEVIRI_2023*_PC.nc"
    nc_files = sorted(glob.glob(loc + nc_filepattern))

    for fpath in tqdm(nc_files, desc=f"Processing {months[key]}"):
        ds = xr.open_dataset(fpath)
        lat_grid = ds['lat'].values  # 1D (665,)
        lon_grid = ds['lon'].values  # 1D (958,)
        satellite_name = ds.EPCT_product_name.split('-')[0]
        timestamp = ds.EPCT_product_name.split('A-')[1].split('.')[0]

        for site_name, (site_lat, site_lon) in sites.items():
            iy, ix = get_crop_center_indices_1d(lat_grid, lon_grid, site_lat, site_lon)

            if iy - half < 0 or iy + half > lat_grid.shape[0] or ix - half < 0 or ix + half > lon_grid.shape[0]:
                print(f"Skipping {fpath} for {site_name}: crop out of bounds")
                continue

            try:
                for i, ch_key in enumerate(channel_keys, start=9):
                    radiances = ds[ch_key].values
                    bt_crop = radiances_2_brightnesstemp_and_reflectances(
                        radiances[iy - half: iy + half, ix - half: ix + half], i, satellite_name
                    )
                    if bt_crop.shape != (crop_size, crop_size):
                        continue
                    all_crops[site_name][i].append(bt_crop)

                # Slice and mesh lat/lon
                lat_crop = lat_grid[iy - half: iy + half]
                lon_crop = lon_grid[ix - half: ix + half]
                #lat_2d, lon_2d = np.meshgrid(lat_crop, lon_crop, indexing='ij')
                all_lats[site_name].append(lat_crop)
                all_lons[site_name].append(lon_crop)
                all_times[site_name].append(timestamp)

            except Exception as e:
                print(f"Error processing {fpath} for {site_name}: {e}")
                continue

        ds.close()
'''
# Write to NetCDFs
    
for site in sites:
    print(f"Writing data for site: {site}")

    # Convert to arrays
    crops_np = {ch: np.array(all_crops[site][ch]) for ch in range(9, 10)}
    lats_np = np.array(all_lats[site])  # Shape: (num_samples, crop_size)
    lons_np = np.array(all_lons[site])  # Shape: (num_samples, crop_size)
    times_np = np.array(all_times[site])  # Shape: (num_samples,)

    num_samples = crops_np[9].shape[0]
    crop_size = crops_np[9].shape[1]

    # Print shape for debug
    print(f"{site} crop shape (channel 9):", crops_np[9].shape)

    # Construct dataset
    data_vars = {
        f"sample_{site}_data": (["sample", "y", "x"], crops_np[ch])
        for ch in range(9, 10)
    }

    ds_out = xr.Dataset(
        data_vars=data_vars,
        coords={
            "sample": np.arange(num_samples),
            "lat": (["sample", "y"], lats_np),
            "lon": (["sample", "x"], lons_np),
            "time": (["sample"], times_np)
        }
    )

    out_path = os.path.join(output_dir, f"msgobs_{site}crops.nc")
    ds_out.to_netcdf(out_path, mode="w")
    print(f"Saved: {out_path}")

Writing data for site: aurillac
aurillac crop shape (channel 9): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_aurillaccrops.nc
Writing data for site: dresden
dresden crop shape (channel 9): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_dresdencrops.nc


In [12]:
d = xr.open_dataset('/p/project1/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_aurillaccrops.nc')
d

<xarray.Dataset>
Dimensions:               (sample: 18567, y: 128, x: 128)
Coordinates:
  * sample                (sample) int64 0 1 2 3 4 ... 18563 18564 18565 18566
    lat                   (sample, y) float64 ...
    lon                   (sample, x) float64 ...
    time                  (sample) object ...
Dimensions without coordinates: y, x
Data variables:
    sample_aurillac_data  (sample, y, x) float32 ...

## Name of datasets

In [ ]:
dataset_obs = {
    'msgobs_108_juelichcrops.nc',
    'msgobs_108_lincrops.nc',
    'msgobs_108_warsawcrops.nc',
    'msgobs_108_viennacrops.nc',
    'msgobs_108_bourgescrops.nc',
    'msgobs_108_zargozacrops.nc',
}

dataset_icon = {
    'msgobs_108_juelichcrops_icon.nc',
    'msgobs_108_lincrops_icon.nc',
    'msgobs_108_warsawcrops_icon.nc',
    'msgobs_108_viennacrops_icon.nc',
    'msgobs_108_bourgescrops_icon.nc',
    'msgobs_108_zaragozacrops_icon.nc',
}

## NC to HDF5

In [14]:
def convert_nc_to_hdf5(nc_file, hdf5_file):
    # Open the NetCDF file using xarray
    ds = xr.open_dataset(nc_file)

    # Create the HDF5 file
    with h5py.File(hdf5_file, 'w') as hdf5_data:
        # Iterate over all variables in the xarray dataset
        for var_name in ds.data_vars:
            var_data = ds[var_name].values
            
            # Create a dataset in the HDF5 file
            hdf5_dataset = hdf5_data.create_dataset(
                var_name, 
                data=var_data, 
                dtype=var_data.dtype, 
                #chunks=chunking,  # Enable chunking if requested
                #compression=compression  # Apply compression if provided
            )

            # Copy variable attributes of the variable to the HDF5 dataset
            for attr_name, attr_value in ds[var_name].attrs.items():
                hdf5_dataset.attrs[attr_name] = attr_value
        
        # Iterate over all coordinates in the xarray dataset
        for coord_name in ds.coords:
            coord_data = ds[coord_name].values
            
            # Handle special case for time coordinate with dtype('O')
            if coord_data.dtype == 'O':
                # Convert to fixed-length strings
                coord_data = coord_data.astype('S')
            
            # Create a dataset in the HDF5 file for the coordinate
            hdf5_coord = hdf5_data.create_dataset(
                coord_name, 
                data=coord_data, 
                dtype=coord_data.dtype, 
                #chunks=chunking,  # Enable chunking if requested
                #compression=compression  # Apply compression if provided
            )
            
            # Copy coordinate attributes to the HDF5 dataset
            for attr_name, attr_value in ds[coord_name].attrs.items():
                hdf5_coord.attrs[attr_name] = attr_value
        
        # Copy global attributes
        for attr_name, attr_value in ds.attrs.items():
            hdf5_data.attrs[attr_name] = attr_value
    # Close the xarray dataset
    ds.close()


nc_file = '/p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_108_aurillaccrops.nc'
hdf5_file = '/p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_108_aurillaccrops.h5'
convert_nc_to_hdf5(
    nc_file, 
    hdf5_file, 
    #chunking=False, 
    #compression=None
)
print(f"Converted {nc_file} to {hdf5_file}")

Converted /p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_108_aurillaccrops.nc to /p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_108_aurillaccrops.h5


# For testing of the quality of clustering we need auxillary data and hence we are extracting the data from fixed locations as a testing data

In [4]:

nc_file_loc = '/p/scratch/exaww/chatterjee1/msg_warmworld/files/'
log_file = nc_file_loc + "processed_files_log.txt"
nan_crop_file = nc_file_loc + "nan_files_log.txt"
output_dir = "/p/project/exaww/chatterjee1/dataset/warmworld_datasets/"
os.makedirs(output_dir, exist_ok=True)

# Months
months = {
    4: '04/', 5: '05/', 6: '06/', 7: '07/', 8: '08/', 9: '09/',
}

# Parameters
channel_keys = [f"channel_{i}" for i in range(4, 12)]  # Channels 4-11
crop_size = 128
half = crop_size // 2

# Site locations
sites = {
    #"juelich": (50.9224, 6.3639),
    #"lindenberg": (52.210, 14.122),
    #"warsaw": (52.229, 21.012),
    #"vienna": (48.2081, 16.3713),
    #"bourges": (47.0812, 2.3980),
    #"zaragoza": (41.6474, -0.8861),
    #"sirta": (48.717, 2.208),
    #"cabauw": (51.9653, 4.8979),
    "nuremberg": (49.4543, 11.0746),
    "aurillac": (44.9332, 2.4483),
    "dresden": (51.0504, 13.7373),
}

# Initialize storage
all_crops = {site: {ch: [] for ch in range(4, 12)} for site in sites}
all_lats = {site: [] for site in sites}
all_lons = {site: [] for site in sites}
all_times = {site: [] for site in sites}

# Utility functions
def get_crop_center_indices_1d(lat_1d, lon_1d, lat_pt, lon_pt):
    iy = np.abs(lat_1d - lat_pt).argmin()
    ix = np.abs(lon_1d - lon_pt).argmin()
    return iy, ix

def radiances_2_brightnesstemp_and_reflectances(radiances, channel_number, satellite_name):
    satellite = MSG_satellite(satellite_name)
    if 4 <= channel_number < 12:
        return satellite.rad_2_tb(channel_number, radiances)
    else:
        raise ValueError(f"Unsupported channel {channel_number} for {satellite_name}")

# Processing loop
for key in months:
    loc = nc_file_loc + months[key]
    nc_filepattern = "HRSEVIRI_2023*_PC.nc"
    nc_files = sorted(glob.glob(loc + nc_filepattern))

    for fpath in tqdm(nc_files, desc=f"Processing {months[key]}"):
        ds = xr.open_dataset(fpath)
        lat_grid = ds['lat'].values  # 1D (665,)
        lon_grid = ds['lon'].values  # 1D (958,)
        satellite_name = ds.EPCT_product_name.split('-')[0]
        timestamp = ds.EPCT_product_name.split('A-')[1].split('.')[0]

        for site_name, (site_lat, site_lon) in sites.items():
            iy, ix = get_crop_center_indices_1d(lat_grid, lon_grid, site_lat, site_lon)

            if iy - half < 0 or iy + half > lat_grid.shape[0] or ix - half < 0 or ix + half > lon_grid.shape[0]:
                print(f"Skipping {fpath} for {site_name}: crop out of bounds")
                continue

            try:
                for i, ch_key in enumerate(channel_keys, start=4):
                    radiances = ds[ch_key].values
                    bt_crop = radiances_2_brightnesstemp_and_reflectances(
                        radiances[iy - half: iy + half, ix - half: ix + half], i, satellite_name
                    )
                    if bt_crop.shape != (crop_size, crop_size):
                        continue
                    all_crops[site_name][i].append(bt_crop)

                # Slice and mesh lat/lon
                lat_crop = lat_grid[iy - half: iy + half]
                lon_crop = lon_grid[ix - half: ix + half]
                lat_2d, lon_2d = np.meshgrid(lat_crop, lon_crop, indexing='ij')
                all_lats[site_name].append(lat_2d)
                all_lons[site_name].append(lon_2d)
                all_times[site_name].append(timestamp)

            except Exception as e:
                print(f"Error processing {fpath} for {site_name}: {e}")
                continue

        ds.close()

# Write to NetCDFs
    
for site in sites:
    print(f"Writing data for site: {site}")

    # Convert to arrays
    crops_np = {ch: np.array(all_crops[site][ch]) for ch in range(4, 12)}
    lats_np = np.array(all_lats[site])  # Shape: (num_samples, crop_size)
    lons_np = np.array(all_lons[site])  # Shape: (num_samples, crop_size)
    times_np = np.array(all_times[site])  # Shape: (num_samples,)

    num_samples = crops_np[4].shape[0]
    crop_size = crops_np[4].shape[1]

    # Print shape for debug
    print(f"{site} crop shape (channel 4):", crops_np[4].shape)

    # Construct dataset
    data_vars = {
        f"sample_{site}_data_{ch}": (["sample", "y", "x"], crops_np[ch])
        for ch in range(4, 12)
    }

    ds_out = xr.Dataset(
        data_vars=data_vars,
        coords={
            "sample": np.arange(num_samples),
            "lat": (["sample", "y","x"], lats_np),
            "lon": (["sample", "y","x"], lons_np),
            "time": (["sample"], times_np)
        }
    )

    out_path = os.path.join(output_dir, f"msgobs_{site}_allchannelcrops.nc")
    ds_out.to_netcdf(out_path, mode="w")
    print(f"Saved: {out_path}")    

Processing 09/: 100%|██████████| 2880/2880 [08:34<00:00,  5.60it/s]


Writing data for site: nuremberg
nuremberg crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_nuremberg_allchannelcrops.nc
Writing data for site: aurillac
aurillac crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_aurillac_allchannelcrops.nc
Writing data for site: dresden
dresden crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/warmworld_datasets/msgobs_dresden_allchannelcrops.nc


In [8]:
lat_grid.shape

(665,)

In [42]:
d = xr.open_dataset('/p/project/exaww/chatterjee1/dataset/msgobs_juelich_allchannelcrops.nc')
d

<xarray.Dataset>
Dimensions:                 (sample: 18567, y: 128, x: 128)
Coordinates:
  * sample                  (sample) int64 0 1 2 3 4 ... 18563 18564 18565 18566
    lat                     (sample, y, x) float64 ...
    lon                     (sample, y, x) float64 ...
    time                    (sample) object ...
Dimensions without coordinates: y, x
Data variables:
    sample_juelich_data_4   (sample, y, x) float32 ...
    sample_juelich_data_5   (sample, y, x) float32 ...
    sample_juelich_data_6   (sample, y, x) float32 ...
    sample_juelich_data_7   (sample, y, x) float32 ...
    sample_juelich_data_8   (sample, y, x) float32 ...
    sample_juelich_data_9   (sample, y, x) float32 ...
    sample_juelich_data_10  (sample, y, x) float32 ...
    sample_juelich_data_11  (sample, y, x) float32 ...

In [46]:
d.sample_juelich_data_4.shape, d.time.shape, d.lat.shape, d.lon.shape

((18567, 128, 128), (18567,), (18567, 128, 128), (18567, 128, 128))

In [47]:
channel_keys

['channel_4',
 'channel_5',
 'channel_6',
 'channel_7',
 'channel_8',
 'channel_9',
 'channel_10',
 'channel_11']

In [27]:
d.sample_cabauw_data_4[1,:,:].shape

(128, 128)

In [26]:
crops_np[4].shape

(18567, 128, 128)

In [39]:
all_crops['juelich'][4][0].shape

(128, 128)

In [40]:
all_lats['juelich'][4][0].shape

(128,)

In [36]:
lats_np.shape

(18567, 128, 128)

In [37]:
ds = xr.open_dataset(fpath)
ds

<xarray.Dataset>
Dimensions:     (lat: 665, lon: 958)
Coordinates:
  * lat         (lat) float64 30.02 30.06 30.09 30.13 ... 54.91 54.94 54.98
  * lon         (lon) float64 -3.481 -3.444 -3.406 -3.368 ... 32.41 32.44 32.48
Data variables:
    crs         |S1 ...
    channel_1   (lat, lon) float32 ...
    channel_2   (lat, lon) float32 ...
    channel_3   (lat, lon) float32 ...
    channel_4   (lat, lon) float32 ...
    channel_5   (lat, lon) float32 ...
    channel_6   (lat, lon) float32 ...
    channel_7   (lat, lon) float32 ...
    channel_8   (lat, lon) float32 ...
    channel_9   (lat, lon) float32 ...
    channel_10  (lat, lon) float32 ...
    channel_11  (lat, lon) float32 ...
Attributes: (12/45)
    Conventions:                    CF-1.5
    ch01_cal:                       -1.114656032994e+00 2.185600064695e-02
    ch02_cal:                       -1.465775717050e+00 2.874070033431e-02
    ch03_cal:                       -1.211260244250e+00 2.375020086765e-02
    ch04_cal:                       -1.865920103496e-01 3.658666869601e-03
    ch05_cal:                       -4.242236706827e-01 8.318111189856e-03
    ...                             ...
    EPCT_roi:                       custom
    EPCT_sensing_start:             null
    EPCT_sensing_stop:              null
    EPCT_software_name:             DataTailor
    EPCT_software_version:          3.4.0
    EPCT_stretch_method:            null

In [38]:
ds.lat.shape

(665,)

### Trying to get the reflectance values as well as brightness temperature from radiance values

In [3]:
C1 = 1.19104*10**(-5)  # in [mW (cm−1)−4 m-2 sr−1]
C2 = 1.43877  # in [K cm]

CHANNEL_NAME = {"channel_1": "VIS 0.6", 
                "channel_2": "VIS 0.8", 
                "channel_3": "NIR 1.6", 
                "channel_4": "IR 3.9", 
                "channel_5": "WV 6.2", 
                "channel_6": "WV 7.3", 
                "channel_7": "IR 8.7", 
                "channel_8": "IR 9.7 - O3", 
                "channel_9": "IR 10.8", 
                "channel_10": "IR 12.0", 
                "channel_11": "IR 13.4 - CO2", }
# in [cm−1]
VC = {'MSG1': {"channel_4": 2567.330, "channel_5": 1598.103, "channel_6": 1362.081, "channel_7": 1149.069, 
                "channel_8": 1034.343, "channel_9": 930.647, "channel_10": 839.660, "channel_11": 752.387
                }, 
      'MSG2': {"channel_4": 2568.832, "channel_5": 1600.548, "channel_6": 1360.330, "channel_7": 1148.620, 
                "channel_8": 1035.289, "channel_9": 931.700, "channel_10": 836.445, "channel_11": 751.792
                }, 
      'MSG3': {"channel_4": 2547.771, "channel_5": 1595.621, "channel_6": 1360.377, "channel_7": 1148.130, 
                "channel_8": 1034.715, "channel_9": 929.842, "channel_10": 838.659, "channel_11": 750.653
                }, 
      'MSG4': {"channel_4": 2555.280, "channel_5": 1596.080, "channel_6": 1361.748, "channel_7": 1147.433, 
                "channel_8": 1034.851, "channel_9": 931.122, "channel_10": 839.113, "channel_11": 748.585
                }, }
# unitless
ALPHA = {'MSG1': {"channel_4": 0.9956, "channel_5": 0.9962, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, 
         'MSG2': {"channel_4": 0.9954, "channel_5": 0.9963, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, 
         'MSG3': {"channel_4": 0.9915, "channel_5": 0.9960, "channel_6": 0.9991, "channel_7": 0.9996, 
                   "channel_8": 0.9999, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9982
                   }, 
         'MSG4': {"channel_4": 0.9916, "channel_5": 0.9959, "channel_6": 0.9990, "channel_7": 0.9996, 
                   "channel_8": 0.9998, "channel_9": 0.9983, "channel_10": 0.9988, "channel_11": 0.9981
                   }, }
# in [K]
BETA = {'MSG1': {"channel_4": 3.410, "channel_5": 2.218, "channel_6": 0.478, "channel_7": 0.179, ''
                  "channel_8": 0.060, "channel_9": 0.625, "channel_10": 0.397, "channel_11": 0.578
                  },
        'MSG2': {"channel_4": 3.438, "channel_5": 2.185, "channel_6": 0.470, "channel_7": 0.179, 
                  "channel_8": 0.056, "channel_9": 0.640, "channel_10": 0.408, "channel_11": 0.561
                  },
        'MSG3': {"channel_4": 2.9002, "channel_5": 2.0337, "channel_6": 0.4340, "channel_7": 0.1714, 
                  "channel_8": 0.0527, "channel_9": 0.6084, "channel_10": 0.3882, "channel_11": 0.5390
                  },
        'MSG4': {"channel_4": 2.9438, "channel_5": 2.0780, "channel_6": 0.4929, "channel_7": 0.1731, 
                  "channel_8": 0.0597, "channel_9": 0.6256, "channel_10": 0.4002, "channel_11": 0.5635
                  }, }

# %%
#############
############# look up tables for calculating reflectances
#############
# constants taken from website: 
# https://eumetsatspace.atlassian.net/wiki/spaces/DSDT/pages/1537277953/MSG15+radiances+conversion+to+BT+and+Reflectances
# and from https://www-cdn.eumetsat.int/files/2020-04/pdf_msg_seviri_rad2refl.pdf

IRRAD = {'MSG1': {"channel_1": 65.2296, "channel_2": 73.0127, "channel_3": 62.3715},
         'MSG2': {"channel_1": 65.2065, "channel_2": 73.1869, "channel_3": 61.9923},
         'MSG3': {"channel_1": 65.5148, "channel_2": 73.1807, "channel_3": 62.0208}, 
         'MSG4': {"channel_1": 65.2656, "channel_2": 73.1692, "channel_3": 61.9416}, }


# %%
class ir_channel:
    """
    class that calls channel specific constants from look up tables above
    """
    def __init__(self, satellite, channel):

        self.name = CHANNEL_NAME[channel]
        self.vc = VC[satellite][channel]  # wavenumber in [cm−1]
        self.alpha = ALPHA[satellite][channel]  # unitless
        self.beta = BETA[satellite][channel]  # in [K]

class vis_nir_channel:
    def __init__(self, satellite, channel):
        
        self.name = CHANNEL_NAME[channel]
        self.irrad = IRRAD[satellite][channel]  # irradiance at 1AU in [mW·m-2·(cm-1)-1]
        
        
def solar_zenith_angle(t, lon, lat):
    """
    Compute the solar zenith angle (in radians) for a given time and location.
    
    Args:
        t (datetime): UTC datetime object
        lon (float): longitude in degrees (East positive)
        lat (float): latitude in degrees (North positive)
    
    Returns:
        float: solar zenith angle in radians
    """
    # Convert time to UTC if not already
    if t.tzinfo is None:
        t = t.replace(tzinfo=timezone.utc)
    
    day_of_year = t.timetuple().tm_yday
    time_decimal = t.hour + t.minute / 60 + t.second / 3600
    
    # Declination angle (in degrees) using Cooper's approximation
    decl = 23.45 * np.sin(np.deg2rad(360 / 365 * (284 + day_of_year)))

    # Time correction factor (Equation of Time approximation)
    B = 2 * np.pi * (day_of_year - 81) / 364
    EoT = 9.87 * np.sin(2*B) - 7.53 * np.cos(B) - 1.5 * np.sin(B)  # in minutes

    # Local Solar Time (LST)
    LSTM = 15 * round(lon / 15)  # Standard meridian
    TC = 4 * (lon - LSTM) + EoT  # Total correction in minutes
    LST = time_decimal + TC / 60  # Local Solar Time in decimal hours

    # Hour angle (H) in degrees
    H = 15 * (LST - 12)

    # Convert angles to radians
    lat_rad = np.deg2rad(lat)
    decl_rad = np.deg2rad(decl)
    H_rad = np.deg2rad(H)

    # Solar zenith angle
    cos_theta = (np.sin(lat_rad) * np.sin(decl_rad) +
                 np.cos(lat_rad) * np.cos(decl_rad) * np.cos(H_rad))
    
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    zenith_angle_rad = np.arccos(cos_theta)
    return zenith_angle_rad

class MSG_satellite:
    def __init__(self, name):
        self.name =  name

    def _get_channel(self, channel_number):
        if channel_number <= 3:
            return vis_nir_channel(satellite=self.name, channel=f"channel_{channel_number}")
        else:
            return ir_channel(satellite=self.name, channel=f"channel_{channel_number}")

    def rad_2_tb(self, channel_number, radiances):
        channel_consts = self._get_channel(channel_number)
        numerator = C2 * channel_consts.vc
        fraction = C1 * channel_consts.vc**3 / radiances + 1
        denominator = channel_consts.alpha * (np.log(fraction))
        tb = numerator / denominator - channel_consts.beta / channel_consts.alpha
        return tb

    def _d(self, t):
        """Approximate Earth-Sun distance in AU"""
        day_of_year = t.timetuple().tm_yday
        return 1.00014 - 0.01671 * np.cos(np.deg2rad(0.9856 * (day_of_year - 4)))

    def _solar_zenith_angle(self, t, lon, lat):
        return solar_zenith_angle(t, lon, lat)

    def rad_2_refl(self, channel_number, radiances, t, lon, lat):
        if channel_number > 3:
            raise ValueError("Reflectance computation only valid for VIS/NIR channels (1–3).")

        channel_consts = self._get_channel(channel_number)
        solar_zenith_angle = self._solar_zenith_angle(t, lon, lat)

        if np.cos(solar_zenith_angle) <= 0:
            return np.nan  # or 0, depending on handling of night values

        distance_au = self._d(t)

        refl = (np.pi * radiances * distance_au**2) / (channel_consts.irrad * np.cos(solar_zenith_angle))
        return refl

# %%
def radiances_2_brightnesstemp_and_reflectances(radiances, channel_number, satellite_name):
    ## radiances in [mW m−2 sr−1 (cm−1)−1)]
    # TODO: add constraint to channel_number (must be >= 4)

    # access correct satellite 
    satellite = MSG_satellite(satellite_name)
    if channel_number <= 3:
        print("not implemented yet for visible and near-infrared")

    elif channel_number >= 4 and channel_number < 12:
        # get brightness temp fro given channel
        return satellite.rad_2_tb(channel_number, radiances)
    
    else:
        print(f"This channel does not exist for satellite {satellite_name}")
        # TODO: raise exception

In [4]:
nc_file_loc = '/p/scratch/exaww/chatterjee1/msg_warmworld/files/'
log_file = nc_file_loc + "processed_files_log.txt"
nan_crop_file = nc_file_loc + "nan_files_log.txt"
output_dir = "/p/project/exaww/chatterjee1/dataset/"
os.makedirs(output_dir, exist_ok=True)

# Months
months = {
    4: '04/', 5: '05/', 6: '06/', 7: '07/', 8: '08/', 9: '09/',
}

# Parameters
channel_keys = [f"channel_{i}" for i in range(1, 12)]  # Channels 4-11
crop_size = 128
half = crop_size // 2

# Site locations
sites = {
    "juelich": (50.9224, 6.3639),
    "lindenberg": (52.210, 14.122),
    "warsaw": (52.229, 21.012),
    "vienna": (48.2081, 16.3713),
    "bourges": (47.0812, 2.3980),
    "zaragoza": (41.6474, -0.8861),
    "sirta": (48.717, 2.208),
    "cabauw": (51.9653, 4.8979),
}

# Initialize storage
all_crops = {site: {ch: [] for ch in range(1, 12)} for site in sites}
all_lats = {site: [] for site in sites}
all_lons = {site: [] for site in sites}
all_times = {site: [] for site in sites}

# Utility functions
def get_crop_center_indices_1d(lat_1d, lon_1d, lat_pt, lon_pt):
    iy = np.abs(lat_1d - lat_pt).argmin()
    ix = np.abs(lon_1d - lon_pt).argmin()
    return iy, ix

def radiances_2_brightnesstemp_and_reflectances(radiances, channel_number, satellite_name, t, lon, lat):
    satellite = MSG_satellite(satellite_name)

    if 1 <= channel_number <= 3:
        refl = satellite.rad_2_refl(channel_number, radiances, t, lon, lat)
        if np.isscalar(refl):
            refl = np.full_like(radiances, np.nan)  # return 2D NaNs with same shape
        return refl

    elif 4 <= channel_number < 12:
        return satellite.rad_2_tb(channel_number, radiances)

    else:
        raise ValueError(f"Unsupported channel {channel_number} for {satellite_name}")

# Processing loop
for key in months:
    loc = nc_file_loc + months[key]
    nc_filepattern = "HRSEVIRI_2023*_PC.nc"
    nc_files = sorted(glob.glob(loc + nc_filepattern))

    for fpath in tqdm(nc_files, desc=f"Processing {months[key]}"):
        ds = xr.open_dataset(fpath)
        lat_grid = ds['lat'].values  # 1D (665,)
        lon_grid = ds['lon'].values  # 1D (958,)
        satellite_name = ds.EPCT_product_name.split('-')[0]
        timestamp = ds.EPCT_product_name.split('A-')[1].split('.')[0]

        for site_name, (site_lat, site_lon) in sites.items():
            iy, ix = get_crop_center_indices_1d(lat_grid, lon_grid, site_lat, site_lon)

            if iy - half < 0 or iy + half > lat_grid.shape[0] or ix - half < 0 or ix + half > lon_grid.shape[0]:
                print(f"Skipping {fpath} for {site_name}: crop out of bounds")
                continue

            try:
                for i, ch_key in enumerate(channel_keys, start=1):
                    radiances = ds[ch_key].values[iy - half: iy + half, ix - half: ix + half]
                    center_lat = lat_grid[iy]
                    center_lon = lon_grid[ix]
                    bt_crop = radiances_2_brightnesstemp_and_reflectances(
                        radiances, i, satellite_name,
                        t=pd.to_datetime(timestamp[0:12], format="%Y%m%d%H%M"),
                        lon=center_lon, lat=center_lat
                    )
                    
                    #bt_crop = radiances_2_brightnesstemp_and_reflectances(
                    #    radiances, i, satellite_name, t=pd.to_datetime(timestamp[0:12], format="%Y%m%d%H%M"), lon=site_lon, lat=site_lat
                    #)
                    if bt_crop.shape != (crop_size, crop_size):
                        continue
                    all_crops[site_name][i].append(bt_crop)

                # Slice and mesh lat/lon
                lat_crop = lat_grid[iy - half: iy + half]
                lon_crop = lon_grid[ix - half: ix + half]
                lat_2d, lon_2d = np.meshgrid(lat_crop, lon_crop, indexing='ij')
                all_lats[site_name].append(lat_2d)
                all_lons[site_name].append(lon_2d)
                all_times[site_name].append(timestamp)

            except Exception as e:
                print(f"Error processing {fpath} for {site_name}: {e}")
                continue

        ds.close()

# Write to NetCDFs

    
for site in sites:
    print(f"Writing data for site: {site}")

    # Convert to arrays
    crops_np = {ch: np.array(all_crops[site][ch]) for ch in range(1, 12)}
    lats_np = np.array(all_lats[site])  # Shape: (num_samples, crop_size)
    lons_np = np.array(all_lons[site])  # Shape: (num_samples, crop_size)
    times_np = np.array(all_times[site])  # Shape: (num_samples,)

    num_samples = crops_np[4].shape[0]
    crop_size = crops_np[4].shape[1]

    # Print shape for debug
    print(f"{site} crop shape (channel 4):", crops_np[4].shape)

    # Construct dataset
    data_vars = {
        f"sample_{site}_data_{ch}": (["sample", "y", "x"], crops_np[ch])
        for ch in range(1, 12)
    }

    ds_out = xr.Dataset(
        data_vars=data_vars,
        coords={
            "sample": np.arange(num_samples),
            "lat": (["sample", "y","x"], lats_np),
            "lon": (["sample", "y","x"], lons_np),
            "time": (["sample"], times_np)
        }
    )

    out_path = os.path.join(output_dir, f"msgobs_{site}_allchannelcrops.nc")
    ds_out.to_netcdf(out_path, mode="w")
    print(f"Saved: {out_path}")

Processing 09/: 100%|██████████| 2880/2880 [12:39<00:00,  3.79it/s]


Writing data for site: juelich
juelich crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/msgobs_juelich_allchannelcrops.nc
Writing data for site: lindenberg
lindenberg crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/msgobs_lindenberg_allchannelcrops.nc
Writing data for site: warsaw
warsaw crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/msgobs_warsaw_allchannelcrops.nc
Writing data for site: vienna
vienna crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/msgobs_vienna_allchannelcrops.nc
Writing data for site: bourges
bourges crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/msgobs_bourges_allchannelcrops.nc
Writing data for site: zaragoza
zaragoza crop shape (channel 4): (18567, 128, 128)
Saved: /p/project/exaww/chatterjee1/dataset/msgobs_zaragoza_allchannelcrops.nc
Writing data for site: sirta
sirta crop 

In [21]:
timestamp

'20230401001241'

In [22]:
pd.to_datetime(timestamp[0:12], format="%Y%m%d%H%M")

Timestamp('2023-04-01 00:12:00')

In [25]:
bt_crop

array([[237.34775, 237.04462, 237.04462, ..., 254.60675, 254.22511,
        252.81406],
       [237.34775, 237.04462, 237.04462, ..., 253.96996, 253.84213,
        253.84213],
       [236.74043, 236.43521, 236.43521, ..., 251.77591, 251.77591,
        252.1664 ],
       ...,
       [254.60675, 254.60675, 256.61972, ..., 246.4226 , 248.3285 ,
        250.33142],
       [255.11348, 255.11348, 255.74362, ..., 252.29628, 252.81406,
        253.84213],
       [255.11348, 255.11348, 255.74362, ..., 252.29628, 252.81406,
        252.81406]], dtype=float32)